In [4]:
#pip install yt-dlp

In [ ]:
#자막 확인 코드 - 필요시만 확인 두번째 코드만 실행 시키면 됨.
import yt_dlp

def check_subtitles(video_url):
    try:
        with yt_dlp.YoutubeDL({'skip_download': True}) as ydl:
            info = ydl.extract_info(video_url, download=False)
            subtitles = info.get('subtitles', {})
            auto_subs = info.get('automatic_captions', {})
            
            print("\n[INFO] ▶️ 사용자 제공 자막:")
            for lang, tracks in subtitles.items():
                print(f"{lang}: {tracks}")
                
            print("\n[INFO] ⚙️ 자동 생성 자막:")
            for lang, tracks in auto_subs.items():
                print(f"{lang}: {tracks}")
                
            if not subtitles and not auto_subs:
                print("[WARNING] 자막이 없습니다!")
                
    except Exception as e:
        print(f"[ERROR] 자막 조회 실패: {e}")

# 테스트
video_url = "https://www.youtube.com/watch?v=rCcxXxoerjg"
check_subtitles(video_url)

In [ ]:
import os
import yt_dlp
import shutil
import subprocess
import re
from reportlab.platypus import SimpleDocTemplate, Paragraph
from reportlab.lib.pagesizes import A4
from reportlab.lib.styles import getSampleStyleSheet
from reportlab.pdfbase import pdfmetrics
from reportlab.pdfbase.ttfonts import TTFont

# 1️⃣ ffmpeg 경로 설정 (Windows 환경)
ffmpeg_path = os.environ.get("FFMPEG_PATH", r"C:\ffmpeg\bin")
if os.path.exists(ffmpeg_path):
    os.environ["PATH"] += os.pathsep + ffmpeg_path
else:
    print("[ERROR] ffmpeg 경로를 확인해 주세요!")
    exit(1)

# 2️⃣ PDF 한글 폰트 등록
font_path = r"C:\Users\CTE22-213\AppData\Local\Microsoft\Windows\Fonts\NanumGothic.ttf"  # 사용자가 지정한 경로로 변경
pdfmetrics.registerFont(TTFont('NanumGothic', font_path))

# 3️⃣ 유튜브 동영상 형식 및 자막 코드 확인
def list_formats_and_subtitles(video_url):
    try:
        with yt_dlp.YoutubeDL({'skip_download': True}) as ydl:
            info = ydl.extract_info(video_url, download=False)

            print("\n🖼️ [INFO] ▶️ 사용 가능한 영상 형식:")
            for fmt in info['formats']:
                res = fmt.get('resolution', 'Unknown')
                size = fmt.get('filesize', 'Unknown')
                ext = fmt.get('ext', 'N/A')
                print(f"  🎞️ ID: {fmt['format_id']}, 해상도: {res}, 크기: {size} bytes, 형식: {ext}")

            subtitles = info.get('subtitles', {})
            auto_subs = info.get('automatic_captions', {})

            print("\n🔤 [INFO] ▶️ 사용자 제공 자막 언어:")
            for lang, tracks in subtitles.items():
                print(f"  🌐 {lang}: {tracks}")

            print("\n⚙️ [INFO] ▶️ 자동 생성 자막 언어:")
            for lang, tracks in auto_subs.items():
                print(f"  🤖 {lang}: {tracks}")

            if not subtitles and not auto_subs:
                print("⚠️ [WARNING] ▶️ 자막이 없는 영상입니다! (영상은 자막 없이 다운로드됨)")

            return bool(subtitles or auto_subs)

    except Exception as e:
        print(f"❌ [ERROR] ▶️ 형식 및 자막 코드 조회 실패: {e}")
        return False

# 4️⃣ 유튜브 동영상 다운로드 (한국어+영어 자막 포함)
def download_video_with_subtitles(video_url):
    try:
        with yt_dlp.YoutubeDL({'skip_download': True}) as ydl:
            info = ydl.extract_info(video_url, download=False)
            video_title = info.get('title', 'unknown_video')
            safe_title = re.sub(r'[\/:*?"<>|]', '_', video_title.replace(' ', '_'))[:50]

        video_folder = os.path.join("downloads", safe_title)
        os.makedirs(video_folder, exist_ok=True)

        subtitle_folder = os.path.join(video_folder, "subtitles")
        os.makedirs(subtitle_folder, exist_ok=True)

        ydl_opts = {
            'format': 'bestvideo[ext=mp4]+bestaudio[ext=m4a]/best',
            'outtmpl': f'{video_folder}/{safe_title}.%(ext)s',
            'ffmpeg_location': ffmpeg_path,
            'writesubtitles': True,
            'writeautomaticsub': True,
            'subtitleslangs': ['en', 'ko'],
            'subtitlesformat': 'vtt',
            'embedsubs': True,
            'keepvideo': True,
            'postprocessors': [
                {'key': 'FFmpegVideoConvertor', 'preferedformat': 'mp4'},
                {'key': 'FFmpegEmbedSubtitle'}
            ],
            'noplaylist': True
        }

        with yt_dlp.YoutubeDL(ydl_opts) as ydl:
            ydl.download([video_url])
            print(f"\n✅ [INFO] ▶️ 다운로드 완료: {safe_title} (한국어 & 영어 자막 포함)")
        return safe_title

    except Exception as e:
        print(f"❌ [ERROR] ▶️ 다운로드 실패: {video_url}, 오류: {e}")
        return None

# 5️⃣ 자막 파일 백업 및 PDF/TXT 생성
def backup_subtitles_and_generate_files(video_title):
    src_folder = os.path.join("downloads", video_title)
    dest_folder = os.path.join(src_folder, "subtitles")
    os.makedirs(dest_folder, exist_ok=True)

    subtitle_files = [f"{video_title}.ko.vtt", f"{video_title}.en.vtt"]
    backed_up = False

    for file in subtitle_files:
        src_path = os.path.join(src_folder, file)
        dest_path = os.path.join(dest_folder, file)
        if os.path.exists(src_path):
            shutil.copy2(src_path, dest_path)
            print(f"🗂️ [INFO] ▶️ 자막 파일 별도 보관 완료: {dest_path}")
            backed_up = True

            txt_path = os.path.join(dest_folder, f"{file}.txt")
            with open(src_path, 'r', encoding='utf-8') as f:
                content = f.read()
            content = re.sub(r'<[^>]+>', '', content)  # 태그 제거
            content = re.sub(r'\d{2}:\d{2}:\d{2}\.\d{3} --> \d{2}:\d{2}:\d{2}\.\d{3}\n', '', content)  # 시간 제거
            content = re.sub(r'\n+', ' ', content)
            content = re.sub(r'\s+', ' ', content).strip()

            with open(txt_path, 'w', encoding='utf-8') as f:
                f.write(content)
            print(f"📄 [INFO] ▶️ TXT 파일 생성 완료: {txt_path}")

            pdf_path = os.path.join(dest_folder, f"{file}.pdf")
            doc = SimpleDocTemplate(pdf_path, pagesize=A4)
            story = []
            style = getSampleStyleSheet()["BodyText"]
            style.fontName = 'NanumGothic'
            for line in content.split('. '):
                if line.strip():
                    story.append(Paragraph(line.strip(), style))
            doc.build(story)
            print(f"📄 [INFO] ▶️ PDF 파일 생성 완료: {pdf_path}")

    if not backed_up:
        print("⚠️ [WARNING] ▶️ 자막 파일을 찾을 수 없습니다. 자막이 영상에만 포함되었을 수 있습니다.")

# 6️⃣ 다운로드된 자막 파일 확인
def print_subtitle_location(video_title):
    subtitle_folder = os.path.join("downloads", video_title, "subtitles")
    subtitle_files = [
        os.path.join(subtitle_folder, f"{video_title}.ko.vtt"),
        os.path.join(subtitle_folder, f"{video_title}.en.vtt")
    ]
    print("\n🗂️ [INFO] ▶️ 별도로 저장된 자막 파일:")
    for file in subtitle_files:
        if os.path.exists(file):
            print(f"  ✅ {os.path.abspath(file)}")
        else:
            print(f"  ⚠️ {file} 파일이 없습니다")

# 7️⃣ 다운로드된 불필요 파일 제거
def clean_temp_files(video_title):
    video_folder = os.path.join("downloads", video_title)
    temp_files = [
        os.path.join(video_folder, f"{video_title}.f140"),
        os.path.join(video_folder, f"{video_title}.f616")
    ]
    for temp_file in temp_files:
        if os.path.exists(temp_file):
            os.remove(temp_file)
            print(f"🧹 [INFO] ▶️ 불필요한 파일 삭제: {temp_file}")

# 8️⃣ 프로그램 실행
if __name__ == "__main__":
    os.makedirs("downloads", exist_ok=True)
    video_urls = [
        "https://www.youtube.com/watch?v=H82l9VvP35I",
        "https://www.youtube.com/watch?v=qqMXwfxmKZA&t=1237s",
    ]

    for url in video_urls:
        has_subtitles = list_formats_and_subtitles(url)

        if has_subtitles:
            video_title = download_video_with_subtitles(url)

            if video_title:
                backup_subtitles_and_generate_files(video_title)
                print_subtitle_location(video_title)
                clean_temp_files(video_title)
        else:
            print("🛑 [INFO] ▶️ 자막이 없는 영상이므로 다운로드를 건너뜁니다.")
